In [33]:
import pandas as pd
import pymorphy2
import re
from tqdm import tqdm 
import folium
from folium.plugins import MarkerCluster
import plotly.express as px
import networkx as nx
import plotly.graph_objects as go
import numpy as np

tqdm.pandas()

### Для примера рассмотрим один файл весом 130 Мб (на хакатоне файлы делились на года, и каждый весил от 2-5 Гб)

In [34]:
df = pd.read_csv("data/april 1.csv", index_col=False)
print("DF Size:" ,df.shape)
df.head(5)

DF Size: (14996, 46)


,Автор (ы),Author full names,Идентификатор автора(ов),Название документа,Год,Название источника,Том,Выпуск,Статья №,Страница начала,Страница окончания,Количество страниц,Цитирования,DOI,Ссылка,Организации,Авторы организаций,Краткое описание,Ключевые слова автора,Ключевые слова указателя,Номера молекулярных последовательностей,Химические вещества/CAS,Фирменные наименования,Производители,Сведения о финансировании,Текст о финансировании,Пристатейные ссылки,Адрес для корреспонденции,Редакторы,Издатель,Спонсоры,Название конференции,Дата конференции,Место проведения конференции,Код конференции,ISSN,ISBN,CODEN,Идентификатор PubMed,Язык оригинального документа,Сокращенное название источника,Тип документа,СТАДИЯ ПУБЛИКАЦИИ,Open Access (открытый доступ),Источник,EID
0,Liu Y.; Dhakal S...,"Liu, Yanbing (57...",57191173694; 572...,Coal and rock in...,2020,Journal of Intel...,38,4,NaN,3949,3959,10.0,18,10.3233/JIFS-179620,https://www.scop...,Key Laboratory o...,"Liu Y., Key Labo...",The coal-rock in...,coal-rock interf...,Coal; Coal depos...,NaN,NaN,NaN,NaN,Institute of Tib...,This study was p...,"Hosseini N., Eva...",Y. Liu; Key Labo...,NaN,IOS Press BV,NaN,NaN,NaN,NaN,NaN,10641246,NaN,NaN,NaN,English,J. Intelligent F...,Article,Final,NaN,Scopus,2-s2.0-85176458839
1,Gao H.; Lu S.; W...,"Gao, Huanbing (2...",23501051800; 855...,Motion path plan...,2020,Journal of Intel...,38,4,NaN,3773,3782,9.0,15,10.3233/JIFS-179600,https://www.scop...,School of Inform...,"Gao H., School o...",The fuzzy contro...,Fuzzy control; m...,Accident prevent...,NaN,NaN,NaN,NaN,Shandong Importa...,This work was su...,"Gao Q., Liu J.G....",H. Gao; School o...,NaN,IOS Press BV,NaN,NaN,NaN,NaN,NaN,10641246,NaN,NaN,NaN,English,J. Intelligent F...,Article,Final,NaN,Scopus,2-s2.0-85176451924
2,Zhang P.; Tao Z....,"Zhang, Peiwen (5...",58691671400; 552...,An ELECTRE TRI-b...,2020,Journal of Intel...,38,4,NaN,4855,4868,13.0,16,10.3233/JIFS-191540,https://www.scop...,School of Econom...,"Zhang P., School...",The aim of this ...,ELECTRE TRI; Mul...,Decision making;...,NaN,NaN,NaN,NaN,Key Research Pro...,The work was sup...,"Liu P.D., Liu J....",Z. Tao; School o...,NaN,IOS Press BV,NaN,NaN,NaN,NaN,NaN,10641246,NaN,NaN,NaN,English,J. Intelligent F...,Article,Final,NaN,Scopus,2-s2.0-85176410774
3,Wang H.; Yao Y.,"Wang, Hanxu (586...",58691372800; 586...,Semantic orderin...,2020,Journal of Intel...,38,4,NaN,3765,3772,7.0,1,10.3233/JIFS-179599,https://www.scop...,School of Intern...,"Wang H., School ...",With the develop...,fuzzy theory; ma...,Computational li...,NaN,NaN,NaN,NaN,NaN,NaN,"Evans J.A., Acev...",Y. Yao; School o...,NaN,IOS Press BV,NaN,NaN,NaN,NaN,NaN,10641246,NaN,NaN,NaN,English,J. Intelligent F...,Article,Final,NaN,Scopus,2-s2.0-85176438310
4,Li Z.; Xu D.,"Li, Zhi (5860730...",58607305600; 356...,Batik dyeing sim...,2020,Journal of Graphics,41,2,NaN,196,203,7.0,2,10.11996/JG.j.20...,https://www.scop...,School of Inform...,"Li Z., School of...","Batik, a traditi...",batik dyeing; di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"GATYS L A, ECKER...",D. Xu; School of...,NaN,Editorial of Boa...,NaN,NaN,NaN,NaN,NaN,2095302X,NaN,NaN,NaN,Chinese,J. Gr.,Article,Final,NaN,Scopus,2-s2.0-85163777147


In [35]:
df.shape

(14996, 46)

### Уберем колонки которые не несут полезную информацию, и  не нужны для анализа

In [36]:
# Список нужных колонок
columns_to_keep = [
    "Author full names", "Идентификатор автора(ов)", "Название документа", "Год", 
    "Название источника", "Цитирования", "DOI", "Организации", "Авторы организаций", 
    "Краткое описание", "Ключевые слова автора", "Ключевые слова указателя", 
    "Сведения о финансировании", "Текст о финансировании", "Пристатейные ссылки", 
    "Адрес для корреспонденции", "Редакторы", "Издатель", "Спонсоры", "ISSN", 
    "ISBN", "CODEN", "Сокращенное название источника", "Тип документа", 
    "СТАДИЯ ПУБЛИКАЦИИ", "Источник", "EID"
]

# Оставляем только указанные колонки
df = df.loc[:, columns_to_keep]
df.head(5)


,Author full names,Идентификатор автора(ов),Название документа,Год,Название источника,Цитирования,DOI,Организации,Авторы организаций,Краткое описание,Ключевые слова автора,Ключевые слова указателя,Сведения о финансировании,Текст о финансировании,Пристатейные ссылки,Адрес для корреспонденции,Редакторы,Издатель,Спонсоры,ISSN,ISBN,CODEN,Сокращенное название источника,Тип документа,СТАДИЯ ПУБЛИКАЦИИ,Источник,EID
0,"Liu, Yanbing (57...",57191173694; 572...,Coal and rock in...,2020,Journal of Intel...,18,10.3233/JIFS-179620,Key Laboratory o...,"Liu Y., Key Labo...",The coal-rock in...,coal-rock interf...,Coal; Coal depos...,Institute of Tib...,This study was p...,"Hosseini N., Eva...",Y. Liu; Key Labo...,NaN,IOS Press BV,NaN,10641246,NaN,NaN,J. Intelligent F...,Article,Final,Scopus,2-s2.0-85176458839
1,"Gao, Huanbing (2...",23501051800; 855...,Motion path plan...,2020,Journal of Intel...,15,10.3233/JIFS-179600,School of Inform...,"Gao H., School o...",The fuzzy contro...,Fuzzy control; m...,Accident prevent...,Shandong Importa...,This work was su...,"Gao Q., Liu J.G....",H. Gao; School o...,NaN,IOS Press BV,NaN,10641246,NaN,NaN,J. Intelligent F...,Article,Final,Scopus,2-s2.0-85176451924
2,"Zhang, Peiwen (5...",58691671400; 552...,An ELECTRE TRI-b...,2020,Journal of Intel...,16,10.3233/JIFS-191540,School of Econom...,"Zhang P., School...",The aim of this ...,ELECTRE TRI; Mul...,Decision making;...,Key Research Pro...,The work was sup...,"Liu P.D., Liu J....",Z. Tao; School o...,NaN,IOS Press BV,NaN,10641246,NaN,NaN,J. Intelligent F...,Article,Final,Scopus,2-s2.0-85176410774
3,"Wang, Hanxu (586...",58691372800; 586...,Semantic orderin...,2020,Journal of Intel...,1,10.3233/JIFS-179599,School of Intern...,"Wang H., School ...",With the develop...,fuzzy theory; ma...,Computational li...,NaN,NaN,"Evans J.A., Acev...",Y. Yao; School o...,NaN,IOS Press BV,NaN,10641246,NaN,NaN,J. Intelligent F...,Article,Final,Scopus,2-s2.0-85176438310
4,"Li, Zhi (5860730...",58607305600; 356...,Batik dyeing sim...,2020,Journal of Graphics,2,10.11996/JG.j.20...,School of Inform...,"Li Z., School of...","Batik, a traditi...",batik dyeing; di...,NaN,NaN,NaN,"GATYS L A, ECKER...",D. Xu; School of...,NaN,Editorial of Boa...,NaN,2095302X,NaN,NaN,J. Gr.,Article,Final,Scopus,2-s2.0-85163777147


### Возьмем ключевые слова спарсив пдф файл с сайта Лабаратории Физики Высоких Энергий ОИЯИ
https://lhep.jinr.ru/tematicheskij-plan-lfve-2021/
### Для примера я возьму эту статью(на хакатоне парсили все статьи)
https://lhep.jinr.ru/wp-content/uploads/2023/06/02-1-1086-2009_2023.pdf



Изучаемая проблема и основная цель исследований:
Странность в адронной материи и исследование граничных эффектов: исследование стабилизирующих эффектов странности в ядерной материи и свойств легчайших гиперядер; исследование многочастичной динамики в неупругих протон-протонных и протон-ядерных взаимодействиях в области предельной множественности; исследования выхода и спектров мягких фотонов в дейтрон-ядерных и ядро-ядерных взаимодействиях.
Ожидаемые результаты по завершении этапов темы или проектов:
﻿﻿﻿Разрешение вопроса о существовании гиперядра д°Н.
﻿﻿﻿Новые экспериментальные данные о свойствах легчайших гиперядер и проверка экспериментом теоретических
моделей для этих гиперядер.
﻿﻿﻿Новые экспериментальные данные о положении границы стабильности (drip-line) для нейтроно-избыточных легких гиперядер, необходимые для развития теории нейтроно-избыточных гиперядер и моделей их рождения в нецентральных ядро-ядерных взаимодействиях.
﻿﻿﻿Новые экспериментальные данные по фоторождению странности и векторных мезонов (в том числе, содержащих
странные кварки) поляризованными фотонами (вблизи соответствующих порогов).
﻿﻿﻿Измерение энергетических спектров гамма-квантов во взаимодействиях различных ядерных пучков Нуклотрона (от дейтерия до тяжелых ядер) на различных ядерных мишенях с теоретическими предсказаниями в области энергий до нескольких десятков МэВ в зависимости от множественности заряженных и нейтральных частиц, от угла вылета фотонов и проверка различных физических гипотез о механизмах образования "прямых мягких фотонов" в протонных и ядерных взаимодействиях.
﻿﻿﻿Подтверждение (или установление верхней границы) сечений образования новых резонансов, распадающихся на два у-кванта.
﻿﻿﻿Сравнение средних значений поперечной и продольной компоненты импульса заряженных частиц в зависимости от множественности. Определение критической области множественности, при которой обе компоненты становятся
Heba3JHyHMbIMH H VCTAHOBJEHME ee CBA3R C OJaCTbIO THOHHOTO KOHJeHCaTa

1. Набор данных для поиска д'Н в пучке ядер Li. Анализ первых экспериментальных гиперядра д°Н и измерению времени жизни изотопов гиперводорода д°Н и д*н.
данных
ПО поиску
2. Молернизация магнитного спектрометра ГиперНИС (трековая система) за счет добавления плоскостей GEM.
детекторов. Эти детекторы, которые уже частично закуплены и тестируются на установке ГипернИС сотрудниками СФСКЯ, будут интегрированы в эту установку для улучшения точности определения вершины распада гиперядер.
Подготовка проекта по совместными с SRC экспериментами, объединения детекторов, разработка технического проекта спектрометра с двумя магнитами (установки второго магнита, подвод коммуникаций, опор для детекторов),

Ожидаемые результаты по этапам темы или проектам в текущем году:
системы сбора данных (проект и тесты), моделирование для оптимальной геометрии совместных

В рамках сотрудничества с Японией, набор данных на установках LEPS/LEPS2 по фоторождению странности и векторных мезонов (в том числе, содержащих странные кварки) поляризованными фотонами (вблизи соответствующих порогов) и анализ ранее накопленных данных об этих реакциях.
﻿﻿﻿Разработка кремниевых фотоумножителей (SiPM) для компактного спектрометра мягких фотонов на основе кристаллов гадолиний-галлиевый граната (GaGG:Ce). Исследование процентного соотношения компонент Cu-W композитного материала в качестве радиаторов калориметра мягких фотонов. Изготовление на ОАО «Интеграл» (г.
Минск), кремниевых фотоумножителей (SiPM). Разработка и изготовление калибровочного устройства на основе пикосекундного твердотельного лазера совместно с НИИ ПФП (г. Минск).
﻿﻿﻿Участие в работах по моделированию работы калориметра для задачи прямых фотонов при разработке физической программы на установке SPD с поляризованными пучками легких ядер и протонов. Участие в моделировании работы создаваемых поляриметров для экспериментов с поляризованными пучками на ускорительном комплексе ЛФВЭ.

### Делим текст на слова, и переводим слова в форму леммы (в англ языке)

In [37]:
# Инициализация MorphAnalyzer
morph = pymorphy2.MorphAnalyzer()

# Пример текста
text = """
The problem under study and the main purpose of the research:
Strangeness in hadron matter and the study of boundary effects: the study of the stabilizing effects of strangeness in nuclear matter and the properties of the lightest hypernuclei; the study of multiparticle dynamics in inelastic proton-proton and proton-nuclear interactions in the field of extreme multiplicity; studies of the yield and spectra of soft photons in deuteron-nuclear and nucleus-nuclear interactions.
Expected results at the end of the stages of the topic or projects:
Resolution of the question of the existence of a hyperkernel d°N.
New experimental data on the properties of the lightest hypernuclei and experimental verification of theoretical
models for these hypernuclei.
New experimental data on the position of the stability boundary (drip-line) for neutron-excess light hypernuclei, necessary for the development of the theory of neutron-excess hypernuclei and models of their birth in non-central core-nuclear interactions.
New experimental data on the photogeneration of strangeness and vector mesons (including those containing
strange quarks) by polarized photons (near the corresponding thresholds).
Measurement of the energy spectra of gamma quanta in the interactions of various nuclear beams of a Nuclotron (from deuterium to heavy nuclei) on various nuclear targets with theoretical predictions in the field of energies up to several tens of MeV, depending on the multiplicity of charged and neutral particles, on the angle of photon departure, and testing various physical hypotheses about the mechanisms of formation of "direct soft photons" in proton and nuclear interactions.
Confirmation (or establishment of an upper bound) of the cross sections of the formation of new resonances decaying into two y-quanta.
Comparison of the average values of the transverse and longitudinal components of the momentum of charged particles depending on the multiplicity. Determination of the critical area of multiplicity at which both components become
Heba3JHyHMbIMH H VCTAHOBJEHME ee CBA3R C OJaCTbIO THOHHOTO KOHJeHCaTa

1. A data set for searching for D'N in a bundle of Li nuclei. Analysis of the first experimental hypernuclei d°N and measurement of the lifetime of isotopes of hyperhydrogen d°N and d*n.
data
BY search
2. Modernization of the HYPERIS magnetic spectrometer (track system) by adding GEM planes.
detectors. These detectors, which have already been partially purchased and are being tested at the HYPERIS installation by SFSC staff, will be integrated into this installation to improve the accuracy of determining the peak of hypernuclear decay.
Preparation of a project for joint experiments with SRC, combining detectors, development of a technical design for a spectrometer with two magnets (installation of a second magnet, supply of communications, supports for detectors),

Expected results for the stages of the topic or projects in the current year:
data collection systems (project and tests), modeling for optimal geometry of joint

Within the framework of cooperation with Japan, a data set at the LEPS/LEPS2 installations on the photogeneration of strangeness and vector mesons (including those containing strange quarks) by polarized photons (near the corresponding thresholds) and an analysis of previously accumulated data on these reactions.
Development of silicon photomultipliers (SiPM) for a compact soft photon spectrometer based on gadolinium-gallium garnet crystals (GaGG:Ce). Investigation of the percentage ratio of Cu-W components of a composite material as radiators of a soft photon calorimeter. Manufacturing at JSC Integral (Moscow
Minsk), silicon photomultipliers (SiPM). Development and manufacture of a calibration device based on a picosecond solid-state laser in cooperation with the Research Institute of PFP (Minsk).
Participation in the work on modeling the operation of a calorimeter for the problem of direct photons in the development of a physical program on an SPD installation with polarized beams of light nuclei and protons. Participation in modeling the operation of polarimeters being created for experiments with polarized beams at the LFVE accelerator complex."""
# Токенизация текста (разделение на слова)
words = text.split()

# Приведение слов к форме леммы
lemmas = [morph.parse(word)[0].normal_form for word in words]

print(lemmas)


['the', 'problem', 'under', 'study', 'and', 'the', 'main', 'purpose', 'of', 'the', 'research:', 'strangeness', 'in', 'hadron', 'matter', 'and', 'the', 'study', 'of', 'boundary', 'effects:', 'the', 'study', 'of', 'the', 'stabilizing', 'effects', 'of', 'strangeness', 'in', 'nuclear', 'matter', 'and', 'the', 'properties', 'of', 'the', 'lightest', 'hypernuclei;', 'the', 'study', 'of', 'multiparticle', 'dynamics', 'in', 'inelastic', 'proton-proton', 'and', 'proton-nuclear', 'interactions', 'in', 'the', 'field', 'of', 'extreme', 'multiplicity;', 'studies', 'of', 'the', 'yield', 'and', 'spectra', 'of', 'soft', 'photons', 'in', 'deuteron-nuclear', 'and', 'nucleus-nuclear', 'interactions.', 'expected', 'results', 'at', 'the', 'end', 'of', 'the', 'stages', 'of', 'the', 'topic', 'or', 'projects:', 'resolution', 'of', 'the', 'question', 'of', 'the', 'existence', 'of', 'a', 'hyperkernel', 'd°n.', 'new', 'experimental', 'data', 'on', 'the', 'properties', 'of', 'the', 'lightest', 'hypernuclei', 'and'

### Удаляем слова паразиты и повторы

In [38]:
# Список слов-паразитов
stop_words = ['the', 'and', 'of', 'in', 'on', 'at', 'by', 'for', 'to', 'with', 'or', 'a', 'an', 'is', 'are', 'this', 'that', 'as']

# Удаление слов-паразитов и повторений
filtered_words = list(set(word for word in words if word.lower() not in stop_words))

# Результат
print(filtered_words)

['will', 'hypernuclear', 'supports', 'nuclear', 'stability', 'inelastic', 'hypernuclei', 'new', "D'N", 'silicon', 'geometry', 'laser', 'corresponding', 'percentage', '(including', 'boundary', 'hypernuclei.', 'd°N.', 'theory', 'those', 'strange', 'components', 'second', 'neutron-excess', 'development', 'ee', 'predictions', 'THOHHOTO', 'previously', 'compact', 'adding', 'magnets', 'CBA3R', 'searching', 'HYPERIS', 'JSC', '(Moscow', 'about', 'crystals', 'transverse', 'experiments', 'position', 'these', 'cross', 'verification', 'purchased', 'SFSC', 'stabilizing', 'LEPS/LEPS2', 'installations', 'gadolinium-gallium', 'energies', 'cooperation', 'Measurement', 'light', 'system)', 'search', 'extreme', 'MeV,', 'calorimeter.', 'studies', 'gamma', 'testing', '(SiPM)', 'complex.', 'topic', 'data', 'project', 'resonances', 'tens', 'current', 'photogeneration', 'reactions.', 'their', 'detectors),', '(Minsk).', 'framework', 'interactions', 'quarks)', 'partially', 'calorimeter', 'Analysis', 'work', 'mul

#### ['HYPERIS', 'those', 'isotopes', 'models', 'multiplicity.', 'under', 'previously', 'solid-state', 'nucleus-nuclear', 'installation', 'values', 'first', 'projects:', 'program', 'Heba3JHyHMbIMH', 'strangeness', 'CBA3R', 'adding', 'calorimeter.', 'particles,', 'components', 'light', 'SFSC', 'charged', 'these', 'quarks)', 'material', 'testing', 'beams', 'boundary', 'Strangeness', 'question', '2.', 'decay.', 'field', 'energy', 'communications,', '(near', 'optimal', 'compact', 'Measurement', 'thresholds).', 'departure,', 'critical', 'have', '(or', 'Confirmation', 'stability', 'OJaCTbIO', 'effects', 'quanta', 'second', 'Resolution', 'polarimeters', 'reactions.', 'photons', 'Minsk),', 'Expected', 'spectra', 'study', 'Determination', '(GaGG:Ce).', '(including', 'Development', 'calibration', 'stages', 'hypernuclei.', 'gamma', 'direct', 'deuterium', 'which', 'Analysis', 'momentum', 'planes.', 'area', 'photons"', 'H', 'Participation', 'based', 'hadron', 'year:', 'verification', 'mechanisms', 'Comparison', 'detectors),', 'y-quanta.', 'data', 'complex.', 'magnetic', 'be', 'integrated', 'problem', 'already', 'nuclei.', 'staff,', 'tests),', 'targets', 'become', 'd*n.', 'These', 'properties', 'yield', 'results', '"direct', 'determining', 'framework', 'heavy', 'joint', 'spectrometer', 'Research', 'mesons', "D'N", 'main', 'depending', 'effects:', 'set', 'core-nuclear', 'stabilizing', 'soft', 'THOHHOTO', 'hypernuclear', 'interactions', 'non-central', 'both', 'theoretical', 'interactions.', 'necessary', 'two', 'modeling', 'sections', 'angle', 'hypernuclei,', 'existence', 'garnet', 'proton-nuclear', 'KOHJeHCaTa', 'resonances', 'Investigation', '(Moscow', 'VCTAHOBJEHME', 'technical', 'hyperkernel', 'C', 'bundle', 'end', 'development', 'Modernization', 'SPD', 'silicon', 'projects', 'Institute', 'studies', 'supports', 'proton', 'LFVE', 'hypernuclei;', 'their', 'radiators', 'LEPS/LEPS2', 'lifetime', '(SiPM)', 'matter', 'topic', 'tested', 'device', 'search', 'predictions', 'energies', 'been', 'photon', 'Manufacturing', '1.', 'thresholds)', 'picosecond', 'photomultipliers', 'containing', '(SiPM).', 'hyperhydrogen', 'JSC', 'current', 'neutral', 'bound)', 'system)', 'new', 'deuteron-nuclear', 'neutron-excess', 'detectors,', 'will', 'peak', 'project', 'purchased', 'upper', 'multiplicity;', 'd°N.', 'corresponding', 'improve', 'magnet,', 'into', 'about', '(Minsk).', '(drip-line)', 'dynamics', 'experiments', 'crystals', 'laser', 'operation', 'accelerator', 'establishment', 'various', 'up', 'composite', 'manufacture', 'GEM', 'inelastic', 'longitudinal', 'measurement', 'being', 'magnets', 'protons.', 'created', 'hypotheses', 'theory', 'average', 'nuclear', 'strange', 'combining', 'position', 'accuracy', 'design', 'geometry', 'multiparticle', 'd°N', 'tens', 'installations', 'Li', 'birth', '(track', '(installation', 'hypernuclei', 'nuclei)', 'Integral', 'experimental', 'decaying', 'SRC,', 'PFP', 'supply', 'physical', 'New', 'vector', 'purpose', 'MeV,', 'formation', 'Cu-W', 'ratio', 'cross', 'ee', 'research:', 'detectors.', 'gadolinium-gallium', 'calorimeter', 'particles', 'analysis', 'multiplicity', 'collection', 'lightest', 'Within', 'photogeneration', 'partially', 'Japan,', 'polarized', 'cooperation', 'Preparation', 'work', 'Nuclotron', 'systems', '(from', 'accumulated', 'percentage', 'extreme', 'transverse', 'searching', 'several', 'nuclei', '(project', 'proton-proton']


### Уберем слова которые не относятся к физике высоких энергий

In [39]:
# Удаление знаков препинания
words_cleaned = [re.sub(r'[^\w\s]', '', word) for word in words]

# Список слов из физики высоких энергий
physics_terms = {'isotopes', 'models', 'multiplicity', 'particles', 'quarks', 'beams', 
                 'field', 'energy', 'stability', 'effects', 'quanta', 'spectra', 'study', 
                 'hypernuclei', 'gamma', 'momentum', 'hadron', 'mechanisms', 'magnetic', 
                 'nuclei', 'targets', 'properties', 'spectrometer', 'mesons', 'theory', 
                 'nuclear', 'interactions', 'resonances', 'hyperkernel', 'accelerator', 
                 'experiments', 'dynamics', 'protons', 'neutron', 'vector', 'multiparticle'}

# Фильтрация слов
filtered_words = list(set([word for word in words_cleaned if word.lower() in physics_terms]))

print(filtered_words)

['study', 'multiparticle', 'nuclear', 'interactions', 'stability', 'quanta', 'spectra', 'hypernuclei', 'accelerator', 'field', 'vector', 'models', 'theory', 'mechanisms', 'protons', 'momentum', 'effects', 'beams', 'hyperkernel', 'targets', 'mesons', 'particles', 'experiments', 'spectrometer', 'dynamics', 'isotopes', 'magnetic', 'nuclei', 'hadron', 'multiplicity', 'properties', 'gamma', 'quarks', 'resonances', 'energy']


#### ['theory', 'isotopes', 'nuclear', 'hadron', 'models', 'mechanisms', 'multiparticle', 'magnetic', 'hypernuclei', 'targets', 'properties', 'vector', 'spectrometer', 'mesons', 'beams', 'field', 'energy', 'interactions', 'dynamics', 'particles', 'experiments', 'accelerator', 'stability', 'multiplicity', 'effects', 'quanta', 'protons', 'resonances', 'quarks', 'spectra', 'study', 'hyperkernel', 'gamma', 'nuclei', 'momentum']



### Просматриваем еще раз вручную

#### ['hadron', 'nuclear', 'hypernuclei', 'multiparticle', 'interactions', 'field', 'spectra', 'hyperkernel', 'vector', 'mesons', 'quarks', 'gamma', 'quanta', 'beams', 'nuclei', 'particles', 'resonances', 'momentum', 'hypernuclei', 'isotopes', 'magnetic', 'spectrometer', 'experiments', 'accelerator']


### Фильтруем колонки в которых есть упоминание темы

In [40]:
filtered_words = ['hadron', 'nuclear', 'hypernuclei', 'multiparticle', 'interactions', 'field', 'spectra', 'hyperkernel', 'vector', 'mesons', 'quarks', 'gamma', 'quanta', 'beams', 'nuclei', 'particles', 'resonances', 'momentum', 'hypernuclei', 'isotopes', 'magnetic', 'spectrometer', 'experiments', 'accelerator']
filtered_words_set = set(filtered_words)

def count_matches(text):
    try:
        # Проверяем, является ли текст строкой, если нет, приводим к строке
        if not isinstance(text, str):
            return 0
        
        words = text.lower().split()  # Приведение к нижнему регистру и разбиение на слова
        lemmas = [morph.parse(word)[0].normal_form for word in words]
        lemmas = set(lemmas)
        
        # Проверяем каждую лемму на принадлежность множеству
        return sum(lemma in filtered_words_set for lemma in lemmas)
    
    except Exception as e:
        print(f"Error processing text: {text}, Error: {e}")
        return 0

df['word_count'] = (
    df["Название документа"].progress_apply(count_matches) + 
    df["Название источника"].progress_apply(count_matches) + 
    df["Организации"].progress_apply(count_matches) + 
    df["Краткое описание"].progress_apply(count_matches) + 
    df["Ключевые слова автора"].progress_apply(count_matches) + 
    df["Ключевые слова указателя"].progress_apply(count_matches) + 
    df["Текст о финансировании"].progress_apply(count_matches) + 
    df["Сокращенное название источника"].progress_apply(count_matches)
)

100%|██████████| 14996/14996 [00:01<00:00, 9564.96it/s] 


In [41]:
filtered_df = df[df['word_count'] > 5].copy()

filtered_df.drop(columns=['word_count'], inplace=True)

filtered_df.head(5)

,Author full names,Идентификатор автора(ов),Название документа,Год,Название источника,Цитирования,DOI,Организации,Авторы организаций,Краткое описание,Ключевые слова автора,Ключевые слова указателя,Сведения о финансировании,Текст о финансировании,Пристатейные ссылки,Адрес для корреспонденции,Редакторы,Издатель,Спонсоры,ISSN,ISBN,CODEN,Сокращенное название источника,Тип документа,СТАДИЯ ПУБЛИКАЦИИ,Источник,EID
343,"Ren, Shanhong (5...",57200446761; 554...,A new formulatio...,2020,International Jo...,9,10.1002/nme.6289,State Key Labora...,"Ren S., State Ke...",This article pre...,continuous trans...,Composite beams ...,Dalian Science a...,Funding text 1: ...,"Abrate S., Di Sc...",G. Zhao; State K...,NaN,John Wiley and S...,NaN,00295981,NaN,IJNMB,Int. J. Numer. M...,Article,Final,Scopus,2-s2.0-85076908043
762,"Huang, Jingjing ...",57194547691; 720...,Aggregating ther...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,State Key Labora...,"Huang J., State ...",Novel chloride-s...,Displacement; Mi...,Critical micelle...,Applied Basic Re...,We acknowledge t...,Ahmadi Mohammad ...,C. Zhong; State ...,NaN,Elsevier Ltd,NaN,00092509,NaN,CESCA,Chem. Eng. Sci.,Article,Final,Scopus,2-s2.0-85077913161
805,"Shi, Yongheng (5...",57216878674; 559...,Effect of anneal...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,Tianjin Key Labo...,"Shi Y., Tianjin ...",In order to stud...,Amorphous alloy;...,NaN,NaN,NaN,"Liu Y., Li J., C...",L. Zhu; Tianjin ...,NaN,Chinese Mechanic...,NaN,02546051,NaN,JRECD,Jinshu Rechuli,Article,Final,Scopus,2-s2.0-85085073371
1779,"Hou, Yuan (57201...",57201201012; 740...,Effects of axial...,2020,Ironmaking and S...,0,10.1080/03019233...,State Key Labora...,"Hou Y., State Ke...",The effects of a...,axial static mag...,Magnetic field e...,National Natural...,This work was fi...,Bhadeshia H.K.D....,Z. Ren; State Ke...,NaN,Taylor and Franc...,NaN,03019233,NaN,IMKSB,Ironmaking Steel...,Article,Final,Scopus,2-s2.0-85056114517
2770,"Hu, Yanchun (555...",55515232600; 572...,Effects of vacan...,2020,Physica B: Conde...,6,10.1016/j.physb....,School of Physic...,"Hu Y., School of...",Sr2FeMo(1-x)O6 (...,Double perovskit...,Magnetic propert...,National Researc...,This work was su...,"Yan Q., Wu H., L...",Y. Hu; School of...,NaN,Elsevier B.V.,NaN,09214526,NaN,PHYBE,Phys B Condens M...,Article,Final,Scopus,2-s2.0-85078110721


In [42]:
filtered_df.shape

(52, 27)

# 52/14496 - кол во подходящих нам статей

In [43]:
# Увеличиваем лимиты отображения строк и колонок
pd.set_option('display.max_rows', None)  # Показывать все строки
pd.set_option('display.max_columns', 100)  # Показывать все столбцы
pd.set_option('display.width', 10)  # Устанавливаем ширину вывода
pd.set_option('display.max_colwidth', 20)  # Показывать всю ширину содержимого ячеек


### Разделяем поле авторы, и адрес для корреспонденции

In [44]:
# Разделение столбца с именами авторов
authors_split = filtered_df['Author full names'].str.split('; ', expand=True)
authors_split.columns = [f'Author {i+1}' for i in range(authors_split.shape[1])]

# Разделение столбца с организациями по ';' для отдельных организаций
org_split = filtered_df['Адрес для корреспонденции'].str.split('; ', expand=True)
org_split.columns = [f'Поле {i+1}' for i in range(org_split.shape[1])]

final_df = pd.concat([filtered_df, authors_split, org_split], axis=1)
final_df.drop(columns=['Адрес для корреспонденции', 'Author full names', 'Идентификатор автора(ов)'], inplace=True)

final_df.head(10)

,Название документа,Год,Название источника,Цитирования,DOI,Организации,Авторы организаций,Краткое описание,Ключевые слова автора,Ключевые слова указателя,Сведения о финансировании,Текст о финансировании,Пристатейные ссылки,Редакторы,Издатель,Спонсоры,ISSN,ISBN,CODEN,Сокращенное название источника,Тип документа,СТАДИЯ ПУБЛИКАЦИИ,Источник,EID,Author 1,Author 2,Author 3,Author 4,Author 5,Author 6,Author 7,Author 8,Author 9,Author 10,Author 11,Поле 1,Поле 2,Поле 3
343,A new formulatio...,2020,International Jo...,9,10.1002/nme.6289,State Key Labora...,"Ren S., State Ke...",This article pre...,continuous trans...,Composite beams ...,Dalian Science a...,Funding text 1: ...,"Abrate S., Di Sc...",NaN,John Wiley and S...,NaN,00295981,NaN,IJNMB,Int. J. Numer. M...,Article,Final,Scopus,2-s2.0-85076908043,"Ren, Shanhong (5...","Zhao, Guozhong (...",None,None,None,None,None,None,None,None,None,G. Zhao,State Key Labora...,email: zhaogz@dl...
762,Aggregating ther...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,State Key Labora...,"Huang J., State ...",Novel chloride-s...,Displacement; Mi...,Critical micelle...,Applied Basic Re...,We acknowledge t...,Ahmadi Mohammad ...,NaN,Elsevier Ltd,NaN,00092509,NaN,CESCA,Chem. Eng. Sci.,Article,Final,Scopus,2-s2.0-85077913161,"Huang, Jingjing ...","Zhong, Chuanrong...","Yang, Yan (57203...",None,None,None,None,None,None,None,None,C. Zhong,State Key Labora...,email: zhchrong2...
805,Effect of anneal...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,Tianjin Key Labo...,"Shi Y., Tianjin ...",In order to stud...,Amorphous alloy;...,NaN,NaN,NaN,"Liu Y., Li J., C...",NaN,Chinese Mechanic...,NaN,02546051,NaN,JRECD,Jinshu Rechuli,Article,Final,Scopus,2-s2.0-85085073371,"Shi, Yongheng (5...","Zhu, Lihua (5590...","Han, Tianheng (5...","Li, Weizheng (57...",None,None,None,None,None,None,None,L. Zhu,Tianjin Key Labo...,email: zhulihua@...
1779,Effects of axial...,2020,Ironmaking and S...,0,10.1080/03019233...,State Key Labora...,"Hou Y., State Ke...",The effects of a...,axial static mag...,Magnetic field e...,National Natural...,This work was fi...,Bhadeshia H.K.D....,NaN,Taylor and Franc...,NaN,03019233,NaN,IMKSB,Ironmaking Steel...,Article,Final,Scopus,2-s2.0-85056114517,"Hou, Yuan (57201...","Ren, Zhongming (...","Zhang, Zhenqiang...","Xuan, Weidong (5...","Wang, Jiang (572...","Yu, Jianbo (1603...","Ren, Xingfu (565...",None,None,None,None,Z. Ren,State Key Labora...,email: renzm2201...
2770,Effects of vacan...,2020,Physica B: Conde...,6,10.1016/j.physb....,School of Physic...,"Hu Y., School of...",Sr2FeMo(1-x)O6 (...,Double perovskit...,Magnetic propert...,National Researc...,This work was su...,"Yan Q., Wu H., L...",NaN,Elsevier B.V.,NaN,09214526,NaN,PHYBE,Phys B Condens M...,Article,Final,Scopus,2-s2.0-85078110721,"Hu, Yanchun (555...","Wang, Shengjie (...","Wang, Xianwei (7...","Tang, Ying (5721...",None,None,None,None,None,None,None,Y. Hu,School of Physic...,email: yanchunhu...
2828,A review of HTGR...,2020,Nuclear Engineer...,39,10.1016/j.nuceng...,Institute of Nuc...,"Sun Q., Institut...",Graphite dust is...,Coagulation; Dep...,Accidents; Coagu...,National Key R&D...,Funding text 1: ...,"Allelein H.-J., ...",NaN,Elsevier Ltd,NaN,00295493,NaN,NEDEA,Nucl Eng Des,Review,Final,Scopus,2-s2.0-85076672354,"Sun, Qi (5719390...","Peng, Wei (35303...","Yu, Suyuan (7405...","Wang, Kaiyuan (5...",None,None,None,None,None,None,None,W. Peng,Institute of Nuc...,email: pengwei@t...
2958,Effect of strong...,2020,Journal of Alloy...,41,10.1016/j.jallco...,State Key Labora...,"Zhao C., State K...",While there have...,Heat treatment; ...,Aluminum alloys;...,National Natural...,Funding text 1: ...,"Yeh J.W., Chen S...",NaN,Elsevier Ltd,NaN,09258388,NaN,JALCE,J Alloys Compd,Article,Final,Scopus,2-s2.0-85076561357,"Zhao, Chendong (...","Li, Jinshan (872...","He, Yixuan (5650...","Wang, Jiaxiang (...","Wang, William Yi...","Kou, Hongchao (5...","Wang, Jun (57200...",None,None,None,None,J. Wang,State Key Labora...,email: nwpuwj

In [45]:
final_df['Кореспондент'] = final_df['Поле 1']
final_df['Адрес'] = final_df['Поле 2']
final_df['Почта'] = final_df['Поле 3']

final_df.drop(columns=['Поле 1', 'Поле 2', 'Поле 3'], inplace=True)

final_df.head(5)

,Название документа,Год,Название источника,Цитирования,DOI,Организации,Авторы организаций,Краткое описание,Ключевые слова автора,Ключевые слова указателя,Сведения о финансировании,Текст о финансировании,Пристатейные ссылки,Редакторы,Издатель,Спонсоры,ISSN,ISBN,CODEN,Сокращенное название источника,Тип документа,СТАДИЯ ПУБЛИКАЦИИ,Источник,EID,Author 1,Author 2,Author 3,Author 4,Author 5,Author 6,Author 7,Author 8,Author 9,Author 10,Author 11,Кореспондент,Адрес,Почта
343,A new formulatio...,2020,International Jo...,9,10.1002/nme.6289,State Key Labora...,"Ren S., State Ke...",This article pre...,continuous trans...,Composite beams ...,Dalian Science a...,Funding text 1: ...,"Abrate S., Di Sc...",NaN,John Wiley and S...,NaN,00295981,NaN,IJNMB,Int. J. Numer. M...,Article,Final,Scopus,2-s2.0-85076908043,"Ren, Shanhong (5...","Zhao, Guozhong (...",None,None,None,None,None,None,None,None,None,G. Zhao,State Key Labora...,email: zhaogz@dl...
762,Aggregating ther...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,State Key Labora...,"Huang J., State ...",Novel chloride-s...,Displacement; Mi...,Critical micelle...,Applied Basic Re...,We acknowledge t...,Ahmadi Mohammad ...,NaN,Elsevier Ltd,NaN,00092509,NaN,CESCA,Chem. Eng. Sci.,Article,Final,Scopus,2-s2.0-85077913161,"Huang, Jingjing ...","Zhong, Chuanrong...","Yang, Yan (57203...",None,None,None,None,None,None,None,None,C. Zhong,State Key Labora...,email: zhchrong2...
805,Effect of anneal...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,Tianjin Key Labo...,"Shi Y., Tianjin ...",In order to stud...,Amorphous alloy;...,NaN,NaN,NaN,"Liu Y., Li J., C...",NaN,Chinese Mechanic...,NaN,02546051,NaN,JRECD,Jinshu Rechuli,Article,Final,Scopus,2-s2.0-85085073371,"Shi, Yongheng (5...","Zhu, Lihua (5590...","Han, Tianheng (5...","Li, Weizheng (57...",None,None,None,None,None,None,None,L. Zhu,Tianjin Key Labo...,email: zhulihua@...
1779,Effects of axial...,2020,Ironmaking and S...,0,10.1080/03019233...,State Key Labora...,"Hou Y., State Ke...",The effects of a...,axial static mag...,Magnetic field e...,National Natural...,This work was fi...,Bhadeshia H.K.D....,NaN,Taylor and Franc...,NaN,03019233,NaN,IMKSB,Ironmaking Steel...,Article,Final,Scopus,2-s2.0-85056114517,"Hou, Yuan (57201...","Ren, Zhongming (...","Zhang, Zhenqiang...","Xuan, Weidong (5...","Wang, Jiang (572...","Yu, Jianbo (1603...","Ren, Xingfu (565...",None,None,None,None,Z. Ren,State Key Labora...,email: renzm2201...
2770,Effects of vacan...,2020,Physica B: Conde...,6,10.1016/j.physb....,School of Physic...,"Hu Y., School of...",Sr2FeMo(1-x)O6 (...,Double perovskit...,Magnetic propert...,National Researc...,This work was su...,"Yan Q., Wu H., L...",NaN,Elsevier B.V.,NaN,09214526,NaN,PHYBE,Phys B Condens M...,Article,Final,Scopus,2-s2.0-85078110721,"Hu, Yanchun (555...","Wang, Shengjie (...","Wang, Xianwei (7...","Tang, Ying (5721...",None,None,None,None,None,None,None,Y. Hu,School of Physic...,email: yanchunhu...


### Рассматриваем таблицу по авторам

In [46]:
authors_publications = pd.melt(
    final_df,
    id_vars=['Название документа', 'Кореспондент', 'Адрес', "Почта" , "Год", 
    "Название источника", "Цитирования", "DOI", "ISSN", 
    "ISBN", "CODEN", "EID"], 
    value_vars=['Author 1', 'Author 2', 'Author 3', 'Author 4', 'Author 5', 'Author 6', 'Author 7', 'Author 8', 'Author 9', 'Author 10', 'Author 11'],
    value_name='author'
)

authors_publications = authors_publications.dropna(subset=['author'])

authors_publications = authors_publications[['author', 'Название документа', 'Кореспондент', 'Адрес', "Почта" , "Год", 
    "Название источника", "Цитирования", "DOI", "ISSN", 
    "ISBN", "CODEN", "EID"]].drop_duplicates()

authors_publications.head(5)

,author,Название документа,Кореспондент,Адрес,Почта,Год,Название источника,Цитирования,DOI,ISSN,ISBN,CODEN,EID
0,"Ren, Shanhong (5...",A new formulatio...,G. Zhao,State Key Labora...,email: zhaogz@dl...,2020,International Jo...,9,10.1002/nme.6289,00295981,NaN,IJNMB,2-s2.0-85076908043
1,"Huang, Jingjing ...",Aggregating ther...,C. Zhong,State Key Labora...,email: zhchrong2...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,00092509,NaN,CESCA,2-s2.0-85077913161
2,"Shi, Yongheng (5...",Effect of anneal...,L. Zhu,Tianjin Key Labo...,email: zhulihua@...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,02546051,NaN,JRECD,2-s2.0-85085073371
3,"Hou, Yuan (57201...",Effects of axial...,Z. Ren,State Key Labora...,email: renzm2201...,2020,Ironmaking and S...,0,10.1080/03019233...,03019233,NaN,IMKSB,2-s2.0-85056114517
4,"Hu, Yanchun (555...",Effects of vacan...,Y. Hu,School of Physic...,email: yanchunhu...,2020,Physica B: Conde...,6,10.1016/j.physb....,09214526,NaN,PHYBE,2-s2.0-85078110721


### Разделяем поле адрес на компоненты

In [47]:
adress_split = authors_publications['Адрес'].str.split(', ', expand=True)
adress_split.columns = [f'Поле {i+1}' for i in range(adress_split.shape[1])]

final_df = pd.concat([authors_publications, adress_split], axis=1)

final_df.drop(columns=['Адрес'], inplace=True)
final_df.head(5)



,author,Название документа,Кореспондент,Почта,Год,Название источника,Цитирования,DOI,ISSN,ISBN,CODEN,EID,Поле 1,Поле 2,Поле 3,Поле 4,Поле 5,Поле 6,Поле 7,Поле 8
0,"Ren, Shanhong (5...",A new formulatio...,G. Zhao,email: zhaogz@dl...,2020,International Jo...,9,10.1002/nme.6289,00295981,NaN,IJNMB,2-s2.0-85076908043,State Key Labora...,Department of En...,Dalian Universit...,Dalian,China,None,None,None
1,"Huang, Jingjing ...",Aggregating ther...,C. Zhong,email: zhchrong2...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,00092509,NaN,CESCA,2-s2.0-85077913161,State Key Labora...,Chengdu Universi...,Chengdu,610059,China,None,None,None
2,"Shi, Yongheng (5...",Effect of anneal...,L. Zhu,email: zhulihua@...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,02546051,NaN,JRECD,2-s2.0-85085073371,Tianjin Key Labo...,Tiangong Univers...,Tianjin,300387,China,None,None,None
3,"Hou, Yuan (57201...",Effects of axial...,Z. Ren,email: renzm2201...,2020,Ironmaking and S...,0,10.1080/03019233...,03019233,NaN,IMKSB,2-s2.0-85056114517,State Key Labora...,Shanghai University,Shanghai,China,None,None,None,None
4,"Hu, Yanchun (555...",Effects of vacan...,Y. Hu,email: yanchunhu...,2020,Physica B: Conde...,6,10.1016/j.physb....,09214526,NaN,PHYBE,2-s2.0-85078110721,School of Physics,Henan Province K...,Henan Normal Uni...,Xinxiang,453007,China,None,None


### Считываем города из полей

In [48]:


# Функция для извлечения почтового индекса или города
def extract_postal_or_city(row):
    # Перебираем поля с 8 до 1
    for i in range(8, 0, -1):
        field = f'Поле {i}'
        value = row[field]
        
        if value is not None:
            if field != f'Поле 0' or field != f'Поле 1':
                value = row[f'Поле {i-1}']  # Если это не пустая строка, возвращаем как предыдущий элемент(город,страна)
                if isinstance(value, str) and value.isdigit():
                    value = row[f'Поле {i-2}'] # Если это число, возвращаем как предыдущий элемент(город,почта,страна)
                return value
            
    return None


final_df['City'] = final_df.apply(extract_postal_or_city, axis=1)
final_df.drop(columns=['Поле 1', 'Поле 2','Поле 3', 'Поле 4','Поле 5', 'Поле 6','Поле 7', 'Поле 8'], inplace=True)
final_df.head(5)

,author,Название документа,Кореспондент,Почта,Год,Название источника,Цитирования,DOI,ISSN,ISBN,CODEN,EID,City
0,"Ren, Shanhong (5...",A new formulatio...,G. Zhao,email: zhaogz@dl...,2020,International Jo...,9,10.1002/nme.6289,00295981,NaN,IJNMB,2-s2.0-85076908043,Dalian
1,"Huang, Jingjing ...",Aggregating ther...,C. Zhong,email: zhchrong2...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,00092509,NaN,CESCA,2-s2.0-85077913161,Chengdu
2,"Shi, Yongheng (5...",Effect of anneal...,L. Zhu,email: zhulihua@...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,02546051,NaN,JRECD,2-s2.0-85085073371,Tianjin
3,"Hou, Yuan (57201...",Effects of axial...,Z. Ren,email: renzm2201...,2020,Ironmaking and S...,0,10.1080/03019233...,03019233,NaN,IMKSB,2-s2.0-85056114517,Shanghai
4,"Hu, Yanchun (555...",Effects of vacan...,Y. Hu,email: yanchunhu...,2020,Physica B: Conde...,6,10.1016/j.physb....,09214526,NaN,PHYBE,2-s2.0-85078110721,Xinxiang


### Датасет города -> координаты

In [49]:
cities = pd.read_csv('cities15000.txt', delimiter='\t', header = None)

In [50]:
cities.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3040051,les Escaldes,les Escaldes,Ehskal'des-Ehndz...,42.50729,1.53414,P,PPLA,AD,NaN,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2024-06-20
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey...",42.50779,1.52109,P,PPLC,AD,NaN,07,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,2020-03-03
2,290503,Warīsān,Warisan,"Warisan,Warsan,W...",25.16744,55.40708,P,PPL,AE,NaN,03,NaN,NaN,NaN,108759,NaN,12,Asia/Dubai,2024-06-11
3,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,...",25.56473,55.55517,P,PPLA,AE,NaN,07,NaN,NaN,NaN,62747,NaN,2,Asia/Dubai,2019-10-24
4,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RA...",25.78953,55.94320,P,PPLA,AE,NaN,05,NaN,NaN,NaN,351943,NaN,2,Asia/Dubai,2019-09-09


### Сравниваем города для получения координат

In [51]:
def is_number(value):
    try:
        pd.to_numeric(value)
        return True
    except ValueError:
        return False

def replace_values(row, cities_df):
    if is_number(row['City']):  # если можно интерпретировать как число
        # ищем число в первой колонке cities
        city_row = cities_df[cities_df[0] == pd.to_numeric(row['City'], errors='coerce')]
        if not city_row.empty:
            return city_row[4].values[0], city_row[5].values[0]  # возвращаем значения из колонок 4 и 5(широта долгота)
        else:
            return None, None  # если не нашли, возвращаем None
    else:
        # если не число, сравниваем с колонкой 1
        city_row = cities_df[cities_df[1] == row['City']]
        if not city_row.empty:
            return city_row[4].values[0], city_row[5].values[0]  # возвращаем значения из колонок 4 и 5(широта долгота)
        else:
            return None, None

final_df[['Latitude', 'Longitude']] = final_df.apply(replace_values, cities_df=cities, axis=1, result_type="expand")


In [52]:
final_df.head(5)

,author,Название документа,Кореспондент,Почта,Год,Название источника,Цитирования,DOI,ISSN,ISBN,CODEN,EID,City,Latitude,Longitude
0,"Ren, Shanhong (5...",A new formulatio...,G. Zhao,email: zhaogz@dl...,2020,International Jo...,9,10.1002/nme.6289,00295981,NaN,IJNMB,2-s2.0-85076908043,Dalian,38.91222,121.60222
1,"Huang, Jingjing ...",Aggregating ther...,C. Zhong,email: zhchrong2...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,00092509,NaN,CESCA,2-s2.0-85077913161,Chengdu,30.66667,104.06667
2,"Shi, Yongheng (5...",Effect of anneal...,L. Zhu,email: zhulihua@...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,02546051,NaN,JRECD,2-s2.0-85085073371,Tianjin,39.14222,117.17667
3,"Hou, Yuan (57201...",Effects of axial...,Z. Ren,email: renzm2201...,2020,Ironmaking and S...,0,10.1080/03019233...,03019233,NaN,IMKSB,2-s2.0-85056114517,Shanghai,31.22222,121.45806
4,"Hu, Yanchun (555...",Effects of vacan...,Y. Hu,email: yanchunhu...,2020,Physica B: Conde...,6,10.1016/j.physb....,09214526,NaN,PHYBE,2-s2.0-85078110721,Xinxiang,35.19033,113.80151


In [53]:
map_df = final_df

In [54]:
def is_numeric(value):
    return isinstance(value, (int, float)) and not pd.isna(value)

# Применяем фильтрацию, чтобы оставить только строки с числовыми значениями для latitude и longitude
map_df = map_df[map_df['Latitude'].apply(is_numeric) & map_df['Longitude'].apply(is_numeric)]

map_df.head(5)


,author,Название документа,Кореспондент,Почта,Год,Название источника,Цитирования,DOI,ISSN,ISBN,CODEN,EID,City,Latitude,Longitude
0,"Ren, Shanhong (5...",A new formulatio...,G. Zhao,email: zhaogz@dl...,2020,International Jo...,9,10.1002/nme.6289,00295981,NaN,IJNMB,2-s2.0-85076908043,Dalian,38.91222,121.60222
1,"Huang, Jingjing ...",Aggregating ther...,C. Zhong,email: zhchrong2...,2020,Chemical Enginee...,14,10.1016/j.ces.20...,00092509,NaN,CESCA,2-s2.0-85077913161,Chengdu,30.66667,104.06667
2,"Shi, Yongheng (5...",Effect of anneal...,L. Zhu,email: zhulihua@...,2020,Jinshu Rechuli/H...,1,10.13251/j.issn....,02546051,NaN,JRECD,2-s2.0-85085073371,Tianjin,39.14222,117.17667
3,"Hou, Yuan (57201...",Effects of axial...,Z. Ren,email: renzm2201...,2020,Ironmaking and S...,0,10.1080/03019233...,03019233,NaN,IMKSB,2-s2.0-85056114517,Shanghai,31.22222,121.45806
4,"Hu, Yanchun (555...",Effects of vacan...,Y. Hu,email: yanchunhu...,2020,Physica B: Conde...,6,10.1016/j.physb....,09214526,NaN,PHYBE,2-s2.0-85078110721,Xinxiang,35.19033,113.80151


In [55]:
# Просчет чистоты каждой записи
location_counts = map_df.groupby(['Latitude', 'Longitude']).size().reset_index(name='frequency')

# Initialize a folium map
m = folium.Map(location=[59.9343, 30.3351], zoom_start=12)

# MarkerCluster object
marker_cluster = MarkerCluster(
    max_cluster_radius=140,
    show_coverage_on_hover=True,
    spiderfy_on_max_zoom=True
).add_to(m)

# Добавления маркеров
for index, row in location_counts.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Frequency: {row['frequency']}",
    ).add_to(marker_cluster)

m.save("frequency_map.html")

In [58]:
plot_df = final_df.head(30)

G = nx.DiGraph()

for _, row in plot_df.iterrows():
    document = row['Название документа']
    author = row['author']
    year = row['Год']
    city = row['City']
    
    # Добавляем узлы
    G.add_node(document, type='document', year=year, city=city)
    G.add_node(author, type='author')
    G.add_node(year, type='year')  # Добавляем год как отдельный узел
    G.add_node(city, type='city')  # Добавляем город как отдельный узел
    
    # Добавляем рёбра
    G.add_edge(year, city)  # Год -> Город
    G.add_edge(city, document)  # Город -> Документ
    G.add_edge(document, author)  # Документ -> Автор

# Позиционирование узлов для радиальной схемы
pos = {}
radius = 1  # Радиус для разных слоёв
angle_step = 2 * np.pi / len(plot_df)  # Шаг угла для равномерного расположения узлов

# Год в центре
pos['Год'] = (0, 0)

# Расположение Городов на первом круге
city_nodes = [node for node in G.nodes if G.nodes[node].get('type') == 'city']
for i, city in enumerate(city_nodes):
    angle = i * angle_step
    pos[city] = (radius * np.cos(angle), radius * np.sin(angle))

# Расположение Документов на втором круге
document_nodes = [node for node in G.nodes if G.nodes[node].get('type') == 'document']
radius += 1  # Увеличиваем радиус для следующего слоя
for i, document in enumerate(document_nodes):
    angle = i * angle_step
    pos[document] = (radius * np.cos(angle), radius * np.sin(angle))

# Расположение Авторов на внешнем круге
author_nodes = [node for node in G.nodes if G.nodes[node].get('type') == 'author']
radius += 1  # Увеличиваем радиус для следующего слоя
for i, author in enumerate(author_nodes):
    angle = i * angle_step
    pos[author] = (radius * np.cos(angle), radius * np.sin(angle))

# Важно: Добавить узлы для годов, если они не были добавлены
for year in plot_df['Год'].unique():
    if year not in pos:  # Добавляем только если год ещё не был добавлен
        pos[year] = (0, 0)

# Извлекаем координаты для рёбер
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)

# Извлекаем координаты для узлов
node_x = []
node_y = []
node_text = []
node_color = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Добавляем текст для узлов
    if G.nodes[node].get('type') == 'document':
        node_text.append(f'{node}\nYear: {G.nodes[node]["year"]}\n{G.nodes[node]["city"]}')
        node_color.append('lightblue')  # Цвет для документов
    elif G.nodes[node].get('type') == 'author':
        node_text.append(node)
        node_color.append('lightgreen')  # Цвет для авторов
    elif G.nodes[node].get('type') == 'year':
        node_text.append(f'Year: {node}')
        node_color.append('lightcoral')  # Цвет для годов
    else:
        node_text.append(f'City: {node}')
        node_color.append('yellowgreen')  # Цвет для городов

# Создаем график
fig = go.Figure()

# Добавляем рёбра
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, mode='lines', line=dict(width=0.5, color='black'),
                         hoverinfo='none'))

# Добавляем вершины с текстом при наведении
fig.add_trace(go.Scatter(x=node_x, y=node_y, mode='markers',
                         text=node_text,  # Текст при наведении
                         hoverinfo='text',  # Текст появляется только при наведении
                         marker=dict(symbol='circle', size=20, color=node_color, line=dict(width=2, color='DarkSlateGrey'))))

# Настройки графика
fig.update_layout(title='Радиальный граф: Год -> Город -> Документ -> Автор',
                  showlegend=False,
                  xaxis=dict(showgrid=False, zeroline=False),
                  yaxis=dict(showgrid=False, zeroline=False),
                  plot_bgcolor='white',
                  hovermode='closest')

# Отображение графика
fig.write_html("interactive graph.html")


In [59]:
# Подсчитываем количество публикаций для каждого автора
author_counts = final_df['author'].value_counts().reset_index()
author_counts.columns = ['author', 'publication_count']

# Сортируем по количеству публикаций и выбираем только 10 лучших авторов
top_10_authors = author_counts.nlargest(10, 'publication_count')

# Создаем barplot с помощью Plotly
fig = px.bar(top_10_authors, x='author', y='publication_count',
             title='Top 10 Authors with Most Publications',
             labels={'author': 'Author', 'publication_count': 'Number of Publications'},
             color='publication_count', color_continuous_scale='Viridis')

# Настроим внешний вид графика
fig.update_layout(showlegend=False,
                  xaxis_title=None,
                  yaxis_title=None,
                  plot_bgcolor='white')


fig.write_html("author_graph.html")